In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage

def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """

    # Base universe set to the QTradableStocksUS
    mask = QTradableStocksUS() & Sector().notnull()
    mask &= (Fundamentals.market_cap.latest > 1000000000)
    mask &= (
        (Fundamentals.morningstar_sector_code.latest != 103) &
        (Fundamentals.morningstar_sector_code.latest != 207) &
        (Fundamentals.morningstar_sector_code.latest != 206) &
        (Fundamentals.morningstar_sector_code.latest != 309) &
        (Fundamentals.morningstar_industry_code.latest != 20533080) &
        (Fundamentals.morningstar_industry_code.latest != 10217033) &
        (Fundamentals.morningstar_industry_group_code != 10106) &
        (Fundamentals.morningstar_industry_group_code != 10104)
    )
    #mask = StaticAssets(symbols(['GOOG', 'AMZN', 'MSFT', 'AAPL', 'DIS', 'TXN', 'SCHW']))
    #mask = StaticAssets(symbols(['TQQQ', 'SPY']))
    
    #mask &= (Fundamentals.pb_ratio.latest < 1.0)
    #mask &= (Fundamentals.current_ratio.latest > 1.0)
    #mask &= (Fundamentals.roic.latest > .15)
    
    earnings_yield = Fundamentals.ebit.latest/Fundamentals.enterprise_value.latest
    fcf_return = Fundamentals.cash_return.latest
    #cf_yield
    roic      = Fundamentals.roic.latest
    debt_equity = Fundamentals.total_debt_equity_ratio.latest
    div_yield = Fundamentals.trailing_dividend_yield.latest
    gross_margin = Fundamentals.gross_margin.latest
    rnd_ocf = Fundamentals.research_and_development.latest / Fundamentals.operating_cash_flow.latest
    total_yield = Fundamentals.total_yield.latest
    quick_ratio = Fundamentals.quick_ratio.latest
    current_ratio = Fundamentals.current_ratio.latest
    
    EY_rank   = earnings_yield.rank(ascending=False, mask=mask)
    FCFR_rank = fcf_return.rank(ascending=False, mask=mask)
    roic_rank = roic          .rank(ascending=False, mask=mask)
    debt_equity_rank = debt_equity.rank(ascending=True, mask=mask)  # lower is better
    div_rank = div_yield.rank(ascending=False, mask=mask)  # higher is better
    gross_margin_rank = gross_margin.rank(ascending=False, mask=mask)
    rnd_ocf_rank = rnd_ocf.rank(ascending=False, mask=mask)
    quick_rank = quick_ratio.rank(ascending=False, mask=mask)
    current_rank = current_ratio.rank(ascending=False, mask=mask)
    
    # dividend and buy back yield
    total_yield_rank = total_yield.rank(ascending=False, mask=mask)
    
    
    ma50 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50, mask=mask)
    ma200 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=200, mask=mask)
    mv_yield = ma50 / ma200
    mv_yield_rank = mv_yield.rank(ascending=False, mask=mask)
    
    sector = Fundamentals.morningstar_sector_code.latest
    
    revenue_growth = Fundamentals.revenue_growth.latest
    revenue_growth_rank = revenue_growth.rank(ascending=False, mask=mask)
    

    
    # Factor of yesterday's close price.
    yesterday_close = USEquityPricing.close.latest

    # Uncomment to include value and rank metric pairs in MF calculation
    columns = {
            'close'         : yesterday_close,
            'sector'        : sector,

            'earnings_yield': earnings_yield,
            'ey_rank'       : EY_rank,
            'roic'          : roic,
            'roic_rank'     : roic_rank,
        
            'debt_equity'   : debt_equity,
            'debt_equity_rank': debt_equity_rank,
            'revenue_growth': revenue_growth,
            'revenue_growth_rank': revenue_growth_rank,
            'fcf_return': fcf_return,
            'fcfr_rank': FCFR_rank,
            #'mv_yield'      : mv_yield,
            #'mv_yield_rank' : mv_yield_rank,
            #'div_yield'     : div_yield,
            #'div_rank'      : div_rank,
            'gross_margin'   : gross_margin,
            'gross_margin_rank': gross_margin_rank,
            'rnd_ocf'       : rnd_ocf,
            'rnd_ocf_rank'  : rnd_ocf_rank,
            #'total_yield'   : total_yield,
            #'total_yield_rank' : total_yield_rank,
            'quick_ratio':   quick_ratio,
            'quick_ratio_rank': quick_rank,
            #'ma50'          : ma50,
            #'ma200'         : ma200,  
        }
    
    mf_rank   = np.sum([rank for key, rank in columns.items() if key[-4:] == 'rank'])
    
    display_columns = {key: value for key, value in columns.items() if key[-4:] != 'rank'}
    display_columns['mf_rank'] = mf_rank
    
    pipe = Pipeline(
        columns=display_columns,
        screen=mask
    )
    return pipe

result = run_pipeline(make_pipeline(), '2020-04-15', '2020-04-15')\
            .sort_values(by='mf_rank', ascending=True)\
            .head(30)#.dropna()
#result['weight'] = np.array(result.reset_index()['mf_rank'].sort_values(ascending=False) / result['mf_rank'].sum())
pd.get_dummies(result)
